In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [22]:
data=pd.read_csv("광주아파트매매데이터.CSV", encoding= "EUC-KR")

In [23]:
data.head(5)

,행정명,번지,본번,부번,단지명,단지명길이,전용면적(㎡),계약년월,계약년,계약월,계약일,거래금액(만원),층,건축년도,건물나이,도로명,해제사유발생일,거래유형,중개사소재지
0,광산구 도산동,1138-2,1138,2,대덕1,3,59.3100,202303,2023,3,18,9800,4,1990,33,송도로 143,NaN,중개거래,광주 광산구
1,광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202302,2023,2,7,21000,2,2000,23,송도로182번길 55,NaN,중개거래,광주 광산구
2,광산구 도산동,790,790,0,대주피오레1차,7,84.9800,202305,2023,5,31,22350,5,2000,23,송도로182번길 55,NaN,중개거래,광주 광산구
3,광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,9,22400,5,2001,22,송도로182번길 16-3,NaN,직거래,-
4,광산구 도산동,820,820,0,대주피오레2차,7,84.7575,202305,2023,5,13,22000,13,2001,22,송도로182번길 16-3,NaN,중개거래,광주 광산구


In [24]:
data=data.drop(['번지','본번','부번','단지명','도로명','해제사유발생일','거래유형','중개사소재지'], axis=1)

In [36]:
data

,행정명,단지명길이,전용면적(㎡),계약년월,계약년,계약월,계약일,거래금액(만원),층,건축년도,건물나이,계약날짜
0,광산구 도산동,3,59.3100,202303,2023,3,18,9800,4,1990,33,2023-03-18
1,광산구 도산동,7,84.9800,202302,2023,2,7,21000,2,2000,23,2023-02-07
2,광산구 도산동,7,84.9800,202305,2023,5,31,22350,5,2000,23,2023-05-31
3,광산구 도산동,7,84.7575,202305,2023,5,9,22400,5,2001,22,2023-05-09
4,광산구 도산동,7,84.7575,202305,2023,5,13,22000,13,2001,22,2023-05-13
...,...,...,...,...,...,...,...,...,...,...,...,...
425074,서구 화정동,4,115.1850,200611,2006,11,10,19000,17,1996,10,2006-11-10
425075,서구 화정동,4,84.8300,200611,2006,11,27,12000,6,1996,10,2006-11-27
425076,서구 화정동,4,59.8300,200611,2006,11,30,7700,2,1996,10,2006-11-30
425077,서구 화정동,4,84.8400,200612,2006,12,6,12170,16,1996,10,2006-12-06


In [32]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 불러오기 (data라는 이름의 데이터프레임으로 가정)
# ... (데이터를 불러오는 코드)

# 계약날짜를 datetime 형식으로 변환
data['계약날짜'] = pd.to_datetime(data['계약년월'].astype(str) + data['계약일'].astype(str), format='%Y%m%d')

# 입력 변수(X)와 타겟 변수(y) 설정
X = data[['계약년','계약월','계약일','단지명길이', '전용면적(㎡)', '층', '건물나이']]  # '계약날짜' 변수를 추가
y = data['거래금액(만원)']

# 행정명 변수를 원핫인코딩하여 더미 변수로 변환
onehot_encoded = pd.get_dummies(data['행정명'])

# X와 onehot_encoded 데이터프레임의 인덱스를 재설정
X.reset_index(drop=True, inplace=True)
onehot_encoded.reset_index(drop=True, inplace=True)

# X에 원핫인코딩된 더미 변수를 추가하여 새로운 X 데이터프레임 생성
X = pd.concat([X, onehot_encoded], axis=1)  # drop_first=True로 첫 번째 카테고리를 제거하여 다중공선성 방지

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 앙상블 모델 학습 (랜덤 포레스트 사용)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
final_prediction = rf_model.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, final_prediction)
r2 = r2_score(y_test, final_prediction)

print("Final Prediction:")
print(final_prediction)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


Final Prediction:
[13808.    6386.6  23338.   ... 20710.5  44498.   15519.75]
Mean Squared Error: 2968458.343977789
R-squared: 0.980846633280767


In [33]:
print("Route Mean Squared Error:", np.sqrt(mse))

Route Mean Squared Error: 1722.921456125551


In [34]:
rf_model.score(X_train,y_train)

0.9971675252983645

In [35]:
rf_model.score(X_test,y_test)

0.980846633280767

In [37]:
X_predict = [[24,12,15,7, 105.5, 14, 5]]
onehot_encoded2 = pd.get_dummies(['동구 학동'])

# X와 onehot_encoded 데이터프레임의 인덱스를 재설정
X_predict.reset_index(drop=True, inplace=True)
onehot_encoded2.reset_index(drop=True, inplace=True)

# X에 원핫인코딩된 더미 변수를 추가하여 새로운 X 데이터프레임 생성
X_predict = pd.concat([X_predict, onehot_encoded], axis=1)

AttributeError: 'list' object has no attribute 'reset_index'

In [39]:
X_predict = np.array([[2024, 8, 1, 3, 59.3100, 4, 33, 0]])  # 행정명 '광산구 도산동'의 인덱스에 해당하는 열은 0으로 설정

# 모델에 입력하여 예측 수행
y_predict = rf_model.predict(X_predict)

print("Predicted Y:", y_predict)

C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


ValueError: X has 8 features, but RandomForestRegressor is expecting 94 features as input.